Assume the data has been loaded from 
http://files.grouplens.org/datasets/movielens/ml-20m.zip and unpacked in data/ml-20m

In [30]:
import scipy.io as sio
import pandas as pd
from scipy.sparse import dok_matrix
from random import sample
import random
random.seed(123)
import graphlab as gl
from graphlab import SFrame
from graphlab import SGraph
import numpy as np
from datetime import datetime
from numpy.linalg import norm

In [235]:
import lib_transform

In [280]:
reload(lib_transform)
from lib_transform import Transformer, get_debug, gen_cs

In [278]:
T = Transformer()

In [155]:
# split normal
train_ids, _, test_ids = sample_split(T.nr)
T.save_ratings_splits_mtx(train_ids, [], test_ids, 'ratings_normal')
T.save_ratings_splits_sf(train_ids, [], test_ids, 'ratings_normal')

In [281]:
# write phi(m) -- features for each movie!
start= datetime.now()
movies_mtx = T.get_movie_matrix(40,"movies.mtx")
print datetime.now()-start

dividing
div 0 1.0
div 10000 1.0
div 20000 0.5
0:11:14.677997


In [ ]:
# split normal debug
debug, debug_train, debug_test, movies_debug = get_debug('ratings')

In [287]:
debug_mids = sample(range(nm), nm / 5)

In [294]:
sio.mmwrite('data/ratings_cs_train.mtx', train_rating_mtx)
sio.mmwrite('data/ratings_cs_test.mtx', test_rating_mtx)

In [295]:
debug_cs, debug_train_cs, debug_test_cs, movie_debug = get_debug('ratings_cs')

sampling
writing debug_train
writing debug_test
movie debug


ValueError: too many values to unpack

In [ ]:
for r in ['','_cs']:
    for d in ['','_debug']:
        for t in ['_train','_test']:
            filename = 'ratings'+r+d+t
            print filename
            coo = sio.mmread('data/%.mtx'%filename)
            get_sf_from_coo(coo, 'data/%s.sgraph'%filename)

In [248]:
ratings_cs_train = sio.mmread('data/ratings_cs_train.mtx').tocsr()
ratings_cs_test = sio.mmread('data/ratings_cs_test.mtx').tocsr()
# debug_mids = sample(range(nm), nm/5)
# debug_uids = sample(range(nu), nu/5)
# debug = ratings_train[debug_uids][:, debug_mids].tocoo()
# debug

In [288]:
movies_read = sio.mmread('data/movies_ratings_debug.mtx').tocsr()
norm(movies_read[0].toarray())  # should be 1

0.99999999999999978

In [289]:
ratings_cs_train, ratings_cs_test, ratings_cs_train + ratings_cs_test

(<138493x27278 sparse matrix of type '<type 'numpy.float64'>'
 	with 15649875 stored elements in Compressed Sparse Row format>,
 <138493x27278 sparse matrix of type '<type 'numpy.float64'>'
 	with 4000052 stored elements in Compressed Sparse Row format>,
 <138493x27278 sparse matrix of type '<type 'numpy.float64'>'
 	with 16519779 stored elements in Compressed Sparse Row format>)

In [251]:
rtraino = ratings_train.tocoo()
rtesto = ratings_test.tocoo()
len(set(zip(rtraino.row, rtraino.col)) & set(zip(rtesto.row, rtesto.col)))

In [298]:
int(5.51)

5